In [ ]:
import sys
sys.path.insert(1, '..')
import local_config
from ovotools import AttrDict
params = AttrDict(data = AttrDict(net_hw = [758,758]))

In [ ]:
import os
import numpy as np

mode = 'train'
data_dir = os.path.join(local_config.data_path, 'DSBI') #My
#BrailleDataset: def __init__(self, params, data_dir, mode):
data_dir_data = os.path.join(data_dir, 'data') # ''
list_file = os.path.join(data_dir, mode + '.txt')
with open(list_file, 'r') as f:
    files = f.readlines()
files = [os.path.join(data_dir_data, fn.replace('.jpg\n', '+recto')) for fn in files]
#files = [os.path.join(data_dir_data, fn.replace('.jpg\n', '')) for fn in files]
assert len(files) > 0, list_file
fn = files[15]
#files 

In [ ]:
import albumentations
import albumentations.augmentations.transforms as T
import albumentations.augmentations.functional as F
import cv2
def common_aug(mode, params):
    '''
    :param mode: 'train', 'test'
    '''
    augs_list = []
    augs_list += [albumentations.RandomCrop(params.data.net_hw[0], params.data.net_hw[1]),]
    #augs_list += [albumentations.Normalize(mean=params.data.mean, std=params.data.std), ]
    return albumentations.Compose(augs_list, p=1.) #, bbox_params = {'format':'albumentations', 'min_visibility':0.5}
aug = common_aug('train', params)

In [ ]:
import PIL.Image
img0 = np.asarray(PIL.Image.open(fn+'.jpg'))
w = 768
img = F.resize(img0, height=int(img0.shape[0]*w/img0.shape[1]), width=w)
tst_w = img.shape[1]//2
img_demo = img[100:100+tst_w//2, 150:100+tst_w]
PIL.Image.fromarray(img_demo)

In [ ]:
aug_img = img
# + rnd. Resize
aug_img = T.ShiftScaleRotate(shift_limit=0, scale_limit=(-0.5,1.0), rotate_limit=(-5,5), border_mode=cv2.BORDER_CONSTANT, always_apply=False, p=1.)(image=aug_img)['image']
aug_img = T.OpticalDistortion(border_mode=cv2.BORDER_CONSTANT, always_apply=False, p=1.)(image=aug_img)['image']
#Crop
aug_img = T.Blur(blur_limit=4, p=1.)(image=aug_img)['image']
aug_img = T.RandomBrightnessContrast(always_apply=False, p=1.)(image=aug_img)['image']
aug_img = T.MotionBlur(always_apply=False, p=1.)(image=aug_img)['image']
aug_img = T.JpegCompression(quality_lower=30, quality_upper=100, always_apply=False, p=1.)(image=aug_img)['image']
#H,V flip

#img_demo = aug_img[100:100+tst_w//2, 150:100+tst_w]
PIL.Image.fromarray(aug_img)

# Normalization (after augmentation)

In [ ]:
import torch
import torchvision.transforms.functional as F

def Normalize(img):
    '''
    returns image converted to FloatTensor and normalized
    '''
    ten_img = F.to_tensor(img)
    means = ten_img.view(3, -1).mean(dim=1)
    std = ten_img.view(3, -1).std(dim=1)
    ten_img = (ten_img-means.view(-1,1,1))/(3*std.view(-1,1,1))
    # decolorize
    ten_img = ten_img.mean(dim=0).expand(3,-1,-1)
    return ten_img

ten_img = Normalize(aug_img)

import numpy as np
def TensorToPilImage(tensor, mean, std):
    vx_np = tensor.cpu().numpy().copy()
    vx_np *= np.asarray(std)[:, np.newaxis, np.newaxis]
    vx_np += np.asarray(mean)[:, np.newaxis, np.newaxis]
    vx_np = vx_np.transpose(1,2,0).clip(0.0, 1.0)*255
    return PIL.Image.fromarray(vx_np.astype(np.uint8))

TensorToPilImage(ten_img, (0.5,), (0.2,))

In [ ]:
import random
import albumentations.augmentations.transforms as T
import albumentations.augmentations.functional as albu_f
import torchvision.transforms.functional as F
import cv2
import albumentations

def common_aug(mode, params):
    '''
    :param mode: 'train', 'test'
    '''
    #aug_params = params.get('augm_params', dict())
    augs_list = []
    assert mode == 'train' # TODO #assert mode in {'train', 'test'}
    if mode == 'train':
        augs_list.append(T.Rotate(limit=5, border_mode=cv2.BORDER_CONSTANT, always_apply=True))
        # augs_list.append(T.OpticalDistortion(border_mode=cv2.BORDER_CONSTANT)) - can't handle boundboxes
        augs_list.append(albumentations.RandomCrop(416, 416, always_apply=True))
        augs_list.append(T.Blur(blur_limit=4))
        augs_list.append(T.RandomBrightnessContrast())
        augs_list.append(T.MotionBlur())
        augs_list.append(T.JpegCompression(quality_lower=30, quality_upper=100))
        augs_list.append(T.VerticalFlip())
        augs_list.append(T.HorizontalFlip())

    return albumentations.Compose(augs_list, p=1., bbox_params = {'format':'albumentations', 'min_visibility':0.5})


def random_resize_and_stretch(img, new_width_range, stretch_limit = 0):
    new_width_range = T.to_tuple(new_width_range)
    stretch_limit = T.to_tuple(stretch_limit, bias=1)
    new_width = int(random.uniform(new_width_range[0], new_width_range[1]))
    stretch = random.uniform(stretch_limit[0], stretch_limit[1])
    new_height = int(img.shape[0]*new_width/img.shape[1]*stretch)
    return albu_f.resize(img, height=new_height, width=new_width, interpolation=cv2.INTER_LINEAR)

aug = common_aug('train', None)
aug_img = random_resize_and_stretch(img,
                                    new_width_range=(614, 1840),
                                    stretch_limit = 0.1)
aug_img = aug(image = aug_img, bboxes = [])['image']

ten_img = Normalize(aug_img)

TensorToPilImage(ten_img, (0.5,), (0.2,))